In [1]:
import timm
import types

model = timm.create_model("vit_small_patch16_384", pretrained=True)

def get_last_selfattention(self, x):
    x = self.patch_embed(x)
    cls_token = self.cls_token.expand(x.shape[0], -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
    if self.dist_token is None:
        x = torch.cat((cls_token, x), dim=1)
    else:
        x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)
    x = self.pos_drop(x + self.pos_embed)
       
    for i, blk in enumerate(self.blocks):
        if i < len(self.blocks) - 1:
            x = blk(x)
        else:
            
            import types
            def forward(self, x, return_attention=False):
                y, attn = self.attn(self.norm1(x))
                if return_attention:
                    return attn
                x = x + self.drop_path(y)
                x = x + self.drop_path(self.mlp(self.norm2(x)))
                return x
            blk.forward = types.MethodType(forward, blk)
            
            def forward(self, x):
                B, N, C = x.shape
                qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
                q, k, v = qkv[0], qkv[1], qkv[2]

                attn = (q @ k.transpose(-2, -1)) * self.scale
                attn = attn.softmax(dim=-1)
                attn = self.attn_drop(attn)

                x = (attn @ v).transpose(1, 2).reshape(B, N, C)
                x = self.proj(x)
                x = self.proj_drop(x)
                return x, attn
            blk.attn.forward = types.MethodType(forward, blk.attn)
            
            # return attention of the last block
            return blk(x, return_attention=True)
        
model.get_last_selfattention = types.MethodType(get_last_selfattention, model)
